![alt text](../img/explore-space-using-python.svg "Over the moon")
# Plan a Moon mission by using Python pandas

In this module, you will:

* Create a clear representation of data from many sources.
* Use Python and pandas to explore data.
* Use data cleansing techniques to get a clear representation of data.
* Hypothesize how much rock sample astronauts might bring back on the Artemis missions.



## Reads Dataset

In [128]:
import pandas as pd
rock_samples = pd.read_csv('data/rocksamples.csv')

# head() shows the first five lines of the DataFrame
rock_samples.head()

,ID,Mission,Type,Subtype,Weight(g),Pristine(%)
0,10001,Apollo11,Soil,Unsieved,125.8,88.36
1,10002,Apollo11,Soil,Unsieved,5629.0,93.73
2,10003,Apollo11,Basalt,Ilmenite,213.0,65.56
3,10004,Apollo11,Core,Unsieved,44.8,71.76
4,10005,Apollo11,Core,Unsieved,53.4,40.31


In [129]:

"""
    Describes the DataFrame
"""
rock_samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2229 entries, 0 to 2228
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           2229 non-null   int64  
 1   Mission      2229 non-null   object 
 2   Type         2229 non-null   object 
 3   Subtype      2226 non-null   object 
 4   Weight(g)    2229 non-null   float64
 5   Pristine(%)  2229 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 104.6+ KB


## Convert Weight from grams to kilogram

In [130]:
rock_samples['Weight(g)'] = rock_samples['Weight(g)'].apply(lambda x : x * 0.001)
rock_samples.rename(columns={'Weight(g)' : 'Weight(Kg)'}, inplace=True)
rock_samples.head()

,ID,Mission,Type,Subtype,Weight(Kg),Pristine(%)
0,10001,Apollo11,Soil,Unsieved,0.1258,88.36
1,10002,Apollo11,Soil,Unsieved,5.6290,93.73
2,10003,Apollo11,Basalt,Ilmenite,0.2130,65.56
3,10004,Apollo11,Core,Unsieved,0.0448,71.76
4,10005,Apollo11,Core,Unsieved,0.0534,40.31


## Mission Dimension

The rock_samples dataframe has a row for every sample that was collected but, we want to understand the rock samples in total as they relate to the specific rockets that brought them back.

In [131]:
missions = pd.DataFrame()
missions['Mission'] = rock_samples['Mission'].unique()
missions.head()


,Mission
0,Apollo11
1,Apollo12
2,Apollo14
3,Apollo15
4,Apollo16


In [132]:
missions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Mission  6 non-null      object
dtypes: object(1)
memory usage: 176.0+ bytes


## Sum total weight by Mission
Adds a new column to the missions dataframe to represent the sum of all samples collected on that mission

In [133]:
sample_total_weight = rock_samples.groupby('Mission')['Weight(Kg)'].sum()
missions = pd.merge(missions, sample_total_weight, on='Mission')
missions.rename(columns={'Weight(Kg)' : 'Sample Weight(Kg)'}, inplace=True)
missions

,Mission,Sample Weight(Kg)
0,Apollo11,21.55424
1,Apollo12,34.34238
2,Apollo14,41.83363
3,Apollo15,75.39910
4,Apollo16,92.46262
5,Apollo17,109.44402


## Get the difference in weights across missions
Take a look at a lot of different cross sections of data that are available. The total weight of the samples increased with each mission, but it's hard to immediately see by how much.

Add one more column to the missions dataframe that simply grabs the difference between the current row and the row preceding it.

In [134]:
missions['Weight Diff'] = missions['Sample Weight(Kg)'].diff()
missions

,Mission,Sample Weight(Kg),Weight Diff
0,Apollo11,21.55424,NaN
1,Apollo12,34.34238,12.78814
2,Apollo14,41.83363,7.49125
3,Apollo15,75.39910,33.56547
4,Apollo16,92.46262,17.06352
5,Apollo17,109.44402,16.98140


## Fills null values with zero

In [135]:
missions['Weight Diff'] = missions['Weight Diff'].fillna(value=0)
missions

,Mission,Sample Weight(Kg),Weight Diff
0,Apollo11,21.55424,0.00000
1,Apollo12,34.34238,12.78814
2,Apollo14,41.83363,7.49125
3,Apollo15,75.39910,33.56547
4,Apollo16,92.46262,17.06352
5,Apollo17,109.44402,16.98140


## Add in command and lunar module data

By using the [NASA Space Science Data Coordinated Archive](https://nssdc.gsfc.nasa.gov/nmc/SpacecraftQuery.jsp), we gathered information about each module used in each mission. As you did when you created the samples tables, create six new columns, three for the lunar modules and three for the command modules:

* Module Name
* Module Mass
* Module Mass Diff

Fill in any NaN values with 0:

In [136]:
"""
    Lunar Modules (LM)
"""
missions['Lunar Module (LM)'] = {
    'Eagle (LM-5)',
    'Interpid (LM-6)',
    'Antares (LM-8)',
    'Falcon (LM-10)',
    'Orion (LM-11)',
    'Challenger (LM-12)'
}

missions['LM Mass (Kg)'] = {
    15103, 
    15235,
    15264,
    16430,
    16445,
    16456
}

missions['LM Mass Diff'] = missions['LM Mass (Kg)'].diff().fillna(value=0)

"""
    Command Modules (CM)
"""
missions['Command Module (CM)'] = {
    'Command (CSM-107)',
    'Yankee Clipper (CM-108)',
    'Kitty Hawk (CM-110)',
    'Endeavor (CM-112)',
    'Casper (CM-113)',
    'America (CM-114)'
}

missions['CM Mass (Kg)'] = {
    5560,
    5609,
    5758,
    5875,
    5840,
    5960
}

missions['CM Mass Diff'] = missions['CM Mass (Kg)'].diff().fillna(value=0)


missions

,Mission,Sample Weight(Kg),Weight Diff,Lunar Module (LM),LM Mass (Kg),LM Mass Diff,Command Module (CM),CM Mass (Kg),CM Mass Diff
0,Apollo11,21.55424,0.00000,Antares (LM-8),15264,0.0,Command (CSM-107),5960,0.0
1,Apollo12,34.34238,12.78814,Eagle (LM-5),15235,-29.0,Casper (CM-113),5609,-351.0
2,Apollo14,41.83363,7.49125,Interpid (LM-6),16456,1221.0,Yankee Clipper (CM-108),5840,231.0
3,Apollo15,75.39910,33.56547,Orion (LM-11),16430,-26.0,Endeavor (CM-112),5875,35.0
4,Apollo16,92.46262,17.06352,Falcon (LM-10),16445,15.0,Kitty Hawk (CM-110),5560,-315.0
5,Apollo17,109.44402,16.98140,Challenger (LM-12),15103,-1342.0,America (CM-114),5758,198.0


## Adding Total Weight Columns

In [137]:
missions['Total Weight (Kg)'] = missions['LM Mass (Kg)'] + missions['CM Mass (Kg)']
missions['Total Weight Diff'] = missions['LM Mass Diff'] + missions['CM Mass Diff']

missions

,Mission,Sample Weight(Kg),Weight Diff,Lunar Module (LM),LM Mass (Kg),LM Mass Diff,Command Module (CM),CM Mass (Kg),CM Mass Diff,Total Weight (Kg),Total Weight Diff
0,Apollo11,21.55424,0.00000,Antares (LM-8),15264,0.0,Command (CSM-107),5960,0.0,21224,0.0
1,Apollo12,34.34238,12.78814,Eagle (LM-5),15235,-29.0,Casper (CM-113),5609,-351.0,20844,-380.0
2,Apollo14,41.83363,7.49125,Interpid (LM-6),16456,1221.0,Yankee Clipper (CM-108),5840,231.0,22296,1452.0
3,Apollo15,75.39910,33.56547,Orion (LM-11),16430,-26.0,Endeavor (CM-112),5875,35.0,22305,9.0
4,Apollo16,92.46262,17.06352,Falcon (LM-10),16445,15.0,Kitty Hawk (CM-110),5560,-315.0,22005,-300.0
5,Apollo17,109.44402,16.98140,Challenger (LM-12),15103,-1342.0,America (CM-114),5758,198.0,20861,-1144.0


## Compare the Data
The interesting thing about predicting how much sample each Artemis mission can bring back is that we don't yet know the full specs of the spacecraft that the Artemis plans on using. Using some information from the [NASA Factsheet on the Space Launch System (SLS) and Orion Modules](https://www.nasa.gov/sites/default/files/atoms/files/0080_sls_fact_sheet_sept2020_09082020_final_0.pdf), we have have data on weights and payloads.

A payload is basically the total amount of weight that a rocket can get up through our atmosphere and into space. So the likelihood that the payload number is more accurate than the exact weights of each module is high, because deciding the payload will likely affect each of the other design decisions.

We know that the Saturn V payload was 43,500 kg, and the weights of the modules varied from mission to mission. So, to determine the ratios that will allow us to make predictions about the Artemis missions, we can use:

* Saturn V payload
* Mission sample weight
* Mission module weight


In [138]:
"""
    Sample to Weight Ratio
"""
saturnVPayload = 43500
missions['Crewed Area : Payload'] = missions['Total Weight (Kg)'] / saturnVPayload
missions['Sample : Crewed Area']  = missions['Sample Weight(Kg)'] / missions['Total Weight (Kg)']
missions['Sample : Payload']      = missions['Sample Weight(Kg)'] / saturnVPayload

missions

,Mission,Sample Weight(Kg),Weight Diff,Lunar Module (LM),LM Mass (Kg),LM Mass Diff,Command Module (CM),CM Mass (Kg),CM Mass Diff,Total Weight (Kg),Total Weight Diff,Crewed Area : Payload,Sample : Crewed Area,Sample : Payload
0,Apollo11,21.55424,0.00000,Antares (LM-8),15264,0.0,Command (CSM-107),5960,0.0,21224,0.0,0.487908,0.001016,0.000495
1,Apollo12,34.34238,12.78814,Eagle (LM-5),15235,-29.0,Casper (CM-113),5609,-351.0,20844,-380.0,0.479172,0.001648,0.000789
2,Apollo14,41.83363,7.49125,Interpid (LM-6),16456,1221.0,Yankee Clipper (CM-108),5840,231.0,22296,1452.0,0.512552,0.001876,0.000962
3,Apollo15,75.39910,33.56547,Orion (LM-11),16430,-26.0,Endeavor (CM-112),5875,35.0,22305,9.0,0.512759,0.003380,0.001733
4,Apollo16,92.46262,17.06352,Falcon (LM-10),16445,15.0,Kitty Hawk (CM-110),5560,-315.0,22005,-300.0,0.505862,0.004202,0.002126
5,Apollo17,109.44402,16.98140,Challenger (LM-12),15103,-1342.0,America (CM-114),5758,198.0,20861,-1144.0,0.479563,0.005246,0.002516


### Save the ratios

We can then use the mean() function to take the average of all those ratios across all the missions. We can then use these ratios to predict the Artemis capacity for samples.



In [139]:
crewedArea_payload_ratio = missions['Crewed Area : Payload'].mean()
sample_crewedAread_ratio = missions['Sample : Crewed Area'].mean()
sample_payload_ratio     = missions['Sample : Payload'].mean()

print(crewedArea_payload_ratio)
print(sample_crewedAread_ratio)
print(sample_payload_ratio)

0.49630268199233724
0.0028946732226251396
0.0014369195019157093


## Predict Artemis Sample Capacity

As mentioned in the preceding unit, we can use the [NASA Factsheet on the Space Launch System (SLS) and Orion Modules](https://www.nasa.gov/sites/default/files/atoms/files/0080_sls_fact_sheet_sept2020_09082020_final_0.pdf) to gather estimated data on the rockets and modules that will be used in the Artemis program.

As a reminder, the [Artemis program](https://www.nasa.gov/specials/artemis/) is NASA's second set of missions to land humans on the surface of the Moon. The program will launch in 2024 and will send not only the next pair of humans, but also the first woman to set foot on the Moon. The preparation for this mission is even bigger than focusing on a Moon landing. It will also provide space for a commercial payload on the ship and is the first step along the [Moon to Mars program](https://www.nasa.gov/topics/moon-to-mars). So, while the Artemis missions will likely bring home additional samples, there are other goals that might affect the amount of capacity that's required to do so.

## Creating an Artemis Mission Dataframe
We don't have all the details about the Artemis mission, but we do know currently that three iterations of the rocket will be cycled through for each mission. Each rocket will have one version meant to sustain a crew and one meant only for cargo. For the purposes of this module, we will focus only on the three rockets meant to house crew, to be more aligned with the Apollo missions. We also know that the expected payload of the Space Launch System (SLS) is expected to grow with each iteration, but that the current weight of Orion (the command and lunar modules combined) has one estimated weight today.

Again, we will call the command and lunar modules the crewed area, and we can create a dataframe with the information we have about the three crewed missions:

In [140]:
artemis_crewed_area = 26520
artemis_mission = pd.DataFrame(
    {
        'Mission' : [
            'artemis1',
            'artemis2',
            'artemis3'
        ],
        'Total Weight (Kg)' : [
            artemis_crewed_area,
            artemis_crewed_area,
            artemis_crewed_area
        ],
        'Payload (Kg)' : [
            26988, 
            37965,
            42955
        ]
    }
)

artemis_mission


,Mission,Total Weight (Kg),Payload (Kg)
0,artemis1,26520,26988
1,artemis2,26520,37965
2,artemis3,26520,42955


And we can estimate the weight of samples based on the ratios we determined from the Artemis missions:

In [141]:
artemis_mission['Sample Weight from Total (Kg)']   = artemis_mission['Total Weight (Kg)'] * sample_crewedAread_ratio
artemis_mission['Sample Weight from Payload (Kg)'] = artemis_mission['Payload (Kg)']      * sample_payload_ratio

artemis_mission

,Mission,Total Weight (Kg),Payload (Kg),Sample Weight from Total (Kg),Sample Weight from Payload (Kg)
0,artemis1,26520,26988,76.766734,38.779584
1,artemis2,26520,37965,76.766734,54.552649
2,artemis3,26520,42955,76.766734,61.722877


Finally, we can get the average of the two predictions:

In [142]:
artemis_mission['Estimated Sample Weight (Kg)'] = (artemis_mission['Sample Weight from Payload (Kg)'] + artemis_mission['Sample Weight from Total (Kg)']) / 2

artemis_mission

,Mission,Total Weight (Kg),Payload (Kg),Sample Weight from Total (Kg),Sample Weight from Payload (Kg),Estimated Sample Weight (Kg)
0,artemis1,26520,26988,76.766734,38.779584,57.773159
1,artemis2,26520,37965,76.766734,54.552649,65.659691
2,artemis3,26520,42955,76.766734,61.722877,69.244806


## Prioritize Moon rock sample gathering based on data

Determining which types of samples to collect from the Moon definitely requires expertise, but we can start to make some assumptions for the purpose of learning how to clean and manipulate data.

First, we can determine how much remains of each sample that was returned from the Apollo missions, given the amount that was originally collected and the percentage of remaining pristine sample.

In [144]:
rock_samples['Remaining (Kg)'] = rock_samples['Weight(Kg)'] * (rock_samples['Pristine(%)'] * .01)

rock_samples.head()

,ID,Mission,Type,Subtype,Weight(Kg),Pristine(%),Remaining (Kg)
0,10001,Apollo11,Soil,Unsieved,0.1258,88.36,0.111157
1,10002,Apollo11,Soil,Unsieved,5.6290,93.73,5.276062
2,10003,Apollo11,Basalt,Ilmenite,0.2130,65.56,0.139643
3,10004,Apollo11,Core,Unsieved,0.0448,71.76,0.032148
4,10005,Apollo11,Core,Unsieved,0.0534,40.31,0.021526


Looking at the ```head()``` or ```info()``` of the rock_samples dataframe isn't actually useful at this point. With over 2,000 samples, it's difficult to get an understanding of what the values are. For that, you can use the ```describe()``` function:

In [145]:
rock_samples.describe()

,ID,Weight(Kg),Pristine(%),Remaining (Kg)
count,2229.000000,2229.000000,2229.000000,2229.000000
mean,52058.432032,0.168253,84.512764,0.138103
std,26207.651471,0.637286,22.057299,0.525954
min,10001.000000,0.000000,0.000000,0.000000
25%,15437.000000,0.003000,80.010000,0.002432
50%,65527.000000,0.010200,92.300000,0.008530
75%,72142.000000,0.093490,98.140000,0.078240
max,79537.000000,11.729000,180.000000,11.169527


This helps us see that, on average, each sample weighs about .16 kg and has about 84% of the original amount remaining. We can use this knowledge to extract only the samples that are likely running low, which means that they have been used a lot by researchers.

In [149]:
low_samples = rock_samples.loc[
    (rock_samples['Weight(Kg)'] >= .16) & 
    (rock_samples['Pristine(%)'] <= 50)
]
low_samples.head()

,ID,Mission,Type,Subtype,Weight(Kg),Pristine(%),Remaining (Kg)
11,10017,Apollo11,Basalt,Ilmenite,0.973,43.71,0.425298
14,10020,Apollo11,Basalt,Ilmenite,0.425,27.88,0.118490
15,10021,Apollo11,Breccia,Regolith,0.250,30.21,0.075525
29,10045,Apollo11,Basalt,Olivine,0.185,12.13,0.022441
37,10057,Apollo11,Basalt,Ilmenite,0.919,35.15,0.323028


In [150]:
low_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 11 to 2183
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              27 non-null     int64  
 1   Mission         27 non-null     object 
 2   Type            27 non-null     object 
 3   Subtype         27 non-null     object 
 4   Weight(Kg)      27 non-null     float64
 5   Pristine(%)     27 non-null     float64
 6   Remaining (Kg)  27 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.7+ KB


Twenty-seven samples seems like a small amount to base a recommendation on. We can probably find some other samples that are needed for more research here on Earth. To discover them, we can use the unique() function to see how many unique types we have across the low_samples and rock_samples dataframes.

In [151]:
low_samples.Type.unique()

array(['Basalt', 'Breccia', 'Soil', 'Core'], dtype=object)

In [152]:
rock_samples.Type.unique()

array(['Soil', 'Basalt', 'Core', 'Breccia', 'Special', 'Crustal'],
      dtype=object)

We can see that, although six unique types were collected across all samples, the samples that are running low are from only four unique types. But this doesn't tell us everything about the samples we might want to focus on. For example, in our low_samples dataframe, how many of each type are actually considered low?

In [154]:
low_samples.groupby('Type')['Weight(Kg)'].count()

Type
Basalt     14
Breccia     8
Core        1
Soil        4
Name: Weight(Kg), dtype: int64

Notice that there are more Basalt and Breccia type rocks with low samples than those of Core and Soil. Additionally, because the likelihood is high that every mission has some Core and Soil collection requirements, we can focus on the Basalt and Breccia rock types for the samples that we need to have collected:

In [155]:
needed_samples = low_samples[
    low_samples['Type'].isin(['Basalt', 'Breccia'])
]
needed_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 11 to 2183
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              22 non-null     int64  
 1   Mission         22 non-null     object 
 2   Type            22 non-null     object 
 3   Subtype         22 non-null     object 
 4   Weight(Kg)      22 non-null     float64
 5   Pristine(%)     22 non-null     float64
 6   Remaining (Kg)  22 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.4+ KB


But are Basalt and Breccia really the only two types of rocks we want to look for?

## Develop a recommendation of Moon rock samples to be collected

Let's take a step back and see how the number of samples compares to the amount of sample. We can compare the total weight from the needed_samples dataframe to the rock_samples dataframe. That is, we'll compare the samples we've identified as running low to all the samples collected on Apollo missions.

In [157]:
needed_samples.groupby('Type')['Weight(Kg)'].sum()

Type
Basalt     17.4234
Breccia    10.1185
Name: Weight(Kg), dtype: float64

In [158]:
rock_samples.groupby('Type')['Weight(Kg)'].sum()

Type
Basalt      93.14077
Breccia    168.88075
Core        19.93587
Crustal      4.74469
Soil        87.58981
Special      0.74410
Name: Weight(Kg), dtype: float64

One bit of information really stands out: we've never had a lot of Crustal rocks in the first place.

We can add Crustal rocks to the set of needed samples:

In [160]:
needed_samples = needed_samples.append(rock_samples.loc[rock_samples['Type'] == 'Crustal' ])
needed_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114 entries, 11 to 2189
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              114 non-null    int64  
 1   Mission         114 non-null    object 
 2   Type            114 non-null    object 
 3   Subtype         114 non-null    object 
 4   Weight(Kg)      114 non-null    float64
 5   Pristine(%)     114 non-null    float64
 6   Remaining (Kg)  114 non-null    float64
dtypes: float64(3), int64(1), object(3)
memory usage: 7.1+ KB


## Summary of needed samples

The final step is to consolidate everything we know into one table that can be shared with the astronauts. First, we need a column for each type of rock that we have already identified as rocks we want more samples of:

In [162]:
needed_samples_overview = pd.DataFrame();
needed_samples_overview['Type'] = needed_samples.Type.unique()
needed_samples_overview

,Type
0,Basalt
1,Breccia
2,Crustal


Next, we want the total weight of each type of rock that was originally collected:

In [164]:
# needed_samples.info()
needed_samples_weight = needed_samples.groupby('Type')['Weight(Kg)'].sum().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, needed_samples_weight, on='Type')
needed_samples_overview.rename(columns={'Weight(Kg)' : 'Total Weight(Kg)'}, inplace=True)

needed_samples_overview

,Type,Total Weight(Kg)
0,Basalt,17.42340
1,Breccia,10.11850
2,Crustal,9.48938


When astronauts are up on the Moon, one way they can identify rocks is by their size. If we can give them an estimated size of each type of rock, that might make their collection process easier.

In [165]:
needed_samples_ave_weights = needed_samples.groupby('Type')['Weight(Kg)'].mean().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, needed_samples_ave_weights, on='Type')
needed_samples_overview.rename(columns={'Weight(Kg)' : 'Ave Weight(Kg)'}, inplace=True)
needed_samples_overview

,Type,Total Weight(Kg),Ave Weight(Kg)
0,Basalt,17.42340,1.244529
1,Breccia,10.11850,1.264812
2,Crustal,9.48938,0.103145


Crustals are small! They're probably a lot harder to spot, so no wonder we don't have a lot of them.

We probably want to give the astronauts some indication of how many of each type we want them to collect. So, for the three types we're looking for, we should grab the total number we have of each type and get the remaining percentage of each type of rock.

In [166]:
total_rock_count = rock_samples.groupby('Type')['ID'].count().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, total_rock_count, on='Type')
needed_samples_overview.rename(columns={'ID':'Number of Samples'}, inplace=True)
total_rocks = needed_samples_overview['Number of Samples'].sum()
needed_samples_overview['Percentage of Rocks'] = needed_samples_overview['Number of Samples'] / total_rocks

needed_samples_overview

,Type,Total Weight(Kg),Ave Weight(Kg),Number of Samples,Percentage of Rocks
0,Basalt,17.42340,1.244529,351,0.258850
1,Breccia,10.11850,1.264812,959,0.707227
2,Crustal,9.48938,0.103145,46,0.033923


And finally, to tie it all back into a recommendation to the Artemis program, we can determine the average weight of samples we estimated in the preceding unit.

In [168]:
artemis_ave_weight = artemis_mission['Estimated Sample Weight (Kg)'].mean()
artemis_ave_weight

64.22588520079607

We can use this number to determine how many of each rock we want the astronauts to aim to collect:

In [169]:
needed_samples_overview['Weight to Collect'] = needed_samples_overview['Percentage of Rocks'] * artemis_ave_weight

needed_samples_overview['Rocks to Collect'] = needed_samples_overview['Weight to Collect'] / needed_samples_overview['Ave Weight(Kg)']

needed_samples_overview

,Type,Total Weight(Kg),Ave Weight(Kg),Number of Samples,Percentage of Rocks,Weight to Collect,Rocks to Collect
0,Basalt,17.42340,1.244529,351,0.258850,16.624842,13.358345
1,Breccia,10.11850,1.264812,959,0.707227,45.422289,35.912271
2,Crustal,9.48938,0.103145,46,0.033923,2.178754,21.123128
